In [26]:
import pandas as pd
import numpy as np

In [60]:
dataset = pd.read_excel('teste.xlsx')
dataset

,Grupo,EST_Codigo,EST_nome,dt_vencimento,path,cod_ped,COD_cpg,sequencial,nota
0,G A,1,G A TERCEIRIZACOES LTDA,2024-09-26,Y:\CELULA FISCAL - FORTES\G.A\Comprovantes pre...,20240900021,2.024090e+10,1.0,NÃO
1,G A,1,G A TERCEIRIZACOES LTDA,2024-09-26,Y:\CELULA FISCAL - FORTES\G.A\RELATORIO DE DES...,20240900021,2.024090e+10,1.0,NÃO
2,G A,1,G A TERCEIRIZACOES LTDA,2024-09-26,Y:\CELULA FISCAL - FORTES\G.A\Comprovantes pre...,20240900022,2.024090e+10,1.0,NÃO
3,G A,1,G A TERCEIRIZACOES LTDA,2024-09-26,Y:\CELULA FISCAL - FORTES\G.A\RELATORIO DE DES...,20240900022,2.024090e+10,1.0,NÃO
4,G A,1,G A TERCEIRIZACOES LTDA,2024-09-26,Y:\CELULA FISCAL - FORTES\G.A\Comprovantes pre...,20240900024,2.024090e+10,1.0,NÃO
...,...,...,...,...,...,...,...,...,...
28340,UNICA,5,UNILOC PRESTACAO DE SERVICOS E APOIO CONDOMINI...,NaT,Y:\CELULA FISCAL - FORTES\UNICA\PAGAMENTOS\IMP...,20250500082,NaN,NaN,SIM
28341,UNICA,5,UNILOC PRESTACAO DE SERVICOS E APOIO CONDOMINI...,NaT,Y:\CELULA FISCAL - FORTES\UNICA\PAGAMENTOS\IMP...,20250500082,NaN,NaN,SIM
28342,UNICA,4,UNICCLA SERVICOS CONDOMINIAIS E LIMPEZA LTDA,NaT,Y:\CELULA FISCAL - FORTES\UNICA\PAGAMENTOS\REC...,20250600128,NaN,NaN,SIM
28343,UNICA,4,UNICCLA SERVICOS CONDOMINIAIS E LIMPEZA LTDA,NaT,Y:\CELULA FISCAL - FORTES\UNICA\PAGAMENTOS\REC...,20250600128,NaN,NaN,SIM


In [65]:
dataset

,Grupo,EST_Codigo,EST_nome,dt_vencimento,path,cod_ped,COD_cpg,sequencial,nota
0,G A,1,G A TERCEIRIZACOES LTDA,2024-09-26,Y:\CELULA FISCAL - FORTES\G.A\Comprovantes pre...,20240900021,2.024090e+10,1.0,NÃO
1,G A,1,G A TERCEIRIZACOES LTDA,2024-09-26,Y:\CELULA FISCAL - FORTES\G.A\RELATORIO DE DES...,20240900021,2.024090e+10,1.0,NÃO
2,G A,1,G A TERCEIRIZACOES LTDA,2024-09-26,Y:\CELULA FISCAL - FORTES\G.A\Comprovantes pre...,20240900022,2.024090e+10,1.0,NÃO
3,G A,1,G A TERCEIRIZACOES LTDA,2024-09-26,Y:\CELULA FISCAL - FORTES\G.A\RELATORIO DE DES...,20240900022,2.024090e+10,1.0,NÃO
4,G A,1,G A TERCEIRIZACOES LTDA,2024-09-26,Y:\CELULA FISCAL - FORTES\G.A\Comprovantes pre...,20240900024,2.024090e+10,1.0,NÃO
...,...,...,...,...,...,...,...,...,...
28340,UNICA,5,UNILOC PRESTACAO DE SERVICOS E APOIO CONDOMINI...,NaT,Y:\CELULA FISCAL - FORTES\UNICA\PAGAMENTOS\IMP...,20250500082,NaN,NaN,SIM
28341,UNICA,5,UNILOC PRESTACAO DE SERVICOS E APOIO CONDOMINI...,NaT,Y:\CELULA FISCAL - FORTES\UNICA\PAGAMENTOS\IMP...,20250500082,NaN,NaN,SIM
28342,UNICA,4,UNICCLA SERVICOS CONDOMINIAIS E LIMPEZA LTDA,NaT,Y:\CELULA FISCAL - FORTES\UNICA\PAGAMENTOS\REC...,20250600128,NaN,NaN,SIM
28343,UNICA,4,UNICCLA SERVICOS CONDOMINIAIS E LIMPEZA LTDA,NaT,Y:\CELULA FISCAL - FORTES\UNICA\PAGAMENTOS\REC...,20250600128,NaN,NaN,SIM


In [76]:
dataset_final = dataset[(dataset['nota'] == 'SIM') | (~dataset['COD_cpg'].isna())]

In [77]:
dataset_final

,Grupo,EST_Codigo,EST_nome,dt_vencimento,path,cod_ped,COD_cpg,sequencial,nota
0,G A,1,G A TERCEIRIZACOES LTDA,2024-09-26,Y:\CELULA FISCAL - FORTES\G.A\Comprovantes pre...,20240900021,2.024090e+10,1.0,NÃO
1,G A,1,G A TERCEIRIZACOES LTDA,2024-09-26,Y:\CELULA FISCAL - FORTES\G.A\RELATORIO DE DES...,20240900021,2.024090e+10,1.0,NÃO
2,G A,1,G A TERCEIRIZACOES LTDA,2024-09-26,Y:\CELULA FISCAL - FORTES\G.A\Comprovantes pre...,20240900022,2.024090e+10,1.0,NÃO
3,G A,1,G A TERCEIRIZACOES LTDA,2024-09-26,Y:\CELULA FISCAL - FORTES\G.A\RELATORIO DE DES...,20240900022,2.024090e+10,1.0,NÃO
4,G A,1,G A TERCEIRIZACOES LTDA,2024-09-26,Y:\CELULA FISCAL - FORTES\G.A\Comprovantes pre...,20240900024,2.024090e+10,1.0,NÃO
...,...,...,...,...,...,...,...,...,...
28340,UNICA,5,UNILOC PRESTACAO DE SERVICOS E APOIO CONDOMINI...,NaT,Y:\CELULA FISCAL - FORTES\UNICA\PAGAMENTOS\IMP...,20250500082,NaN,NaN,SIM
28341,UNICA,5,UNILOC PRESTACAO DE SERVICOS E APOIO CONDOMINI...,NaT,Y:\CELULA FISCAL - FORTES\UNICA\PAGAMENTOS\IMP...,20250500082,NaN,NaN,SIM
28342,UNICA,4,UNICCLA SERVICOS CONDOMINIAIS E LIMPEZA LTDA,NaT,Y:\CELULA FISCAL - FORTES\UNICA\PAGAMENTOS\REC...,20250600128,NaN,NaN,SIM
28343,UNICA,4,UNICCLA SERVICOS CONDOMINIAIS E LIMPEZA LTDA,NaT,Y:\CELULA FISCAL - FORTES\UNICA\PAGAMENTOS\REC...,20250600128,NaN,NaN,SIM
